# Import Movie Review Data

Set the seed

In [4]:
import numpy as np

In [5]:
np.random.seed(42)

Import the dataset as pandas dataframe

In [6]:
import pandas as pd

Data can be downloaded from Kaggle at the following URL

- https://www.kaggle.com/c/word2vec-nlp-tutorial/data

In [8]:
df = pd.read_csv('labeledTrainData.tsv',header=0, delimiter="\t", quoting=3)

Split Data into Training and Test Data

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'],
    df['sentiment'],
    test_size=0.2, 
    random_state=42
)

# Build the Tokenizer

In [14]:
from keras.preprocessing.text import Tokenizer

C:\Users\namo venkatesha\AppData\Local\conda\conda\envs\tfdeeplearning\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [15]:
top_words = 10000

In [16]:
t = Tokenizer(num_words=top_words) # num_words -> Vocablury size

In [17]:
t.fit_on_texts(X_train.tolist())

# Prepare Training and Test Data

Get the word index for each of the word in the review

In [18]:
X_train = t.texts_to_sequences(X_train.tolist())

In [19]:
X_test = t.texts_to_sequences(X_test.tolist())

How many words in each review?

# Pad Sequences - Important

In [21]:
from keras.preprocessing import sequence

In [22]:
max_review_length = 300

In [23]:
X_train = sequence.pad_sequences(X_train,maxlen=max_review_length,padding='post')

In [24]:
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length, padding='post')

# Build Embedding Matrix from Pre-Trained Word2Vec

Load pre-trained Gensim Embeddings

In [25]:
import gensim

C:\Users\namo venkatesha\AppData\Local\conda\conda\envs\tfdeeplearning\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [26]:
word2vec = gensim.models.Word2Vec.load('word2vec-movie-50')

Embedding Size

In [27]:
embedding_vector_length = word2vec.wv.syn0.shape[1]

C:\Users\namo venkatesha\AppData\Local\conda\conda\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


Build matrix for current data

In [28]:
embedding_matrix = np.zeros((top_words + 1, embedding_vector_length))

In [29]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > top_words:
        break
    if word in word2vec.wv.vocab:
        embedding_vector = word2vec.wv[word]
        embedding_matrix[i] = embedding_vector

# Build the Graph

In [30]:
from keras.models import Sequential

ImportError: No module named 'tensorflow.python.keras'

In [ ]:
from keras.layers import Dropout, Dense, Embedding, Flatten, Average, Conv1D

In [ ]:
model = Sequential()

Add Embedding layer

In [ ]:
model.add(Embedding(top_words + 1,
                    embedding_vector_length,
                    input_length=max_review_length,
                   weights=[embedding_matrix],
                   trainable=False)
         )

In [ ]:
model.add(Conv1D(64,3,activation='relu'))

In [ ]:
model.add(Conv1D(32,3,activation='relu'))

In [ ]:
model.add(Dropout(0.5))

In [ ]:
model.add(Conv1D(16,3,activation='relu'))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dropout(0.3))

In [ ]:
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# Execute the graph

In [ ]:
model.fit(X_train,y_train,
          epochs=10,
          batch_size=128,          
          validation_data=(X_test, y_test),
         verbose=1)